Get results by criteria

In [4]:
flipped_list = ['250005-003','250005-004','250005-007','276004-001','348003-002','703001-018','703003-010']

In [7]:
from glob import glob
import nibabel as nib
import numpy as np
from tqdm import tqdm

shape_list = []
path_list = glob('custom_quorum_RV_removed\**\*gt.nii.gz', recursive=True)
for path in tqdm(path_list):
    data = nib.load(path)
    arr = data.get_fdata()
    myshape = arr.shape[:2]
    shape_list.append(myshape)
shape_list = np.stack(shape_list)
print(shape_list.shape)
print(shape_list.mean(0))

100%|██████████| 390/390 [04:54<00:00,  1.33it/s]

(390, 2)
[285.61025641 267.34358974]


In [3]:
%matplotlib qt

import os
import matplotlib.pyplot as plt
from skimage.measure import regionprops
import io
from PIL import Image
import nibabel as nib
import numpy as np

names = [('patient020_frame11', 2), ('patient047_frame09', 4), ('patient085_frame09', 14), ('patient086_frame08', 5), ('patient085_frame01', 14), ('patient086_frame01', 2)]
fig, ax = plt.subplots(len(names), 4, figsize=(7, 8))
#fig, ax = plt.subplots(4, len(names), figsize=(8, 5))
for idx, payload in enumerate(names):
    name, index = payload
    baseline_path_pred = os.path.join(r'ACDC_output\Baseline\cv_niftis_raw', name + '.nii.gz')
    no_transformer_path_pred = os.path.join(r'ACDC_output\no_transformer\cv_niftis_raw', name + '.nii.gz')

    path_gt = os.path.join(r'ACDC_output\Baseline\gt_niftis', name + '.nii.gz')
    path_img = os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\imagesTr', name + '_0000.nii.gz')

    data_baseline = nib.load(baseline_path_pred)
    data_no_transformer = nib.load(no_transformer_path_pred)
    data_gt = nib.load(path_gt)
    data_img = nib.load(path_img)

    baseline_pred = data_baseline.get_fdata()
    no_transformer_pred = data_no_transformer.get_fdata()
    gt = data_gt.get_fdata()
    img = data_img.get_fdata()

    label = np.logical_or(gt[:, :, index] > 0, baseline_pred[:, :, index] > 0)
    label = np.logical_or(label, no_transformer_pred[:, :, index] > 0).astype(np.uint8)
    regions = regionprops(label)
    assert len(regions) == 1
    min_row, min_col, max_row, max_col = regions[0].bbox

    #for j in range(img.shape[-1]):
    #    print(j)
    #    fig, ax = plt.subplots(1, 4)
    #    ax[0].imshow(img[:, :, j], cmap='gray')
    #    ax[1].imshow(gt[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[2].imshow(baseline_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[3].imshow(no_transformer_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    plt.show()
    #    plt.waitforbuttonpress()
    #    plt.close(fig)
    #print('************************************')

    img = img[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    gt = gt[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    baseline_pred = baseline_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    no_transformer_pred = no_transformer_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    
    ax[idx, 0].imshow(img, cmap='gray')
    ax[idx, 0].axis('off')
    ax[idx, 1].imshow(gt, cmap='gray', vmin=0, vmax=3)
    ax[idx, 1].axis('off')
    ax[idx, 2].imshow(baseline_pred, cmap='gray', vmin=0, vmax=3)
    ax[idx, 2].axis('off')
    ax[idx, 3].imshow(no_transformer_pred, cmap='gray', vmin=0, vmax=3)
    ax[idx, 3].axis('off')

    #ax[0, idx].imshow(img, cmap='gray')
    #ax[0, idx].axis('off')
    #ax[1, idx].imshow(gt, cmap='gray', vmin=0, vmax=3)
    #ax[1, idx].axis('off')
    #ax[2, idx].imshow(baseline_pred, cmap='gray', vmin=0, vmax=3)
    #ax[2, idx].axis('off')
    #ax[3, idx].imshow(no_transformer_pred, cmap='gray', vmin=0, vmax=3)
    #ax[3, idx].axis('off')

    plt.tight_layout()
#plt.savefig("visual_ablation_study_2.png", dpi=600)

png1 = io.BytesIO()
plt.savefig(png1, format="png", dpi=600)

png2 = Image.open(png1)

png2.save("Image_5.tiff", dpi=(600, 600))
png1.close()

In [12]:
%matplotlib qt

import os
import matplotlib.pyplot as plt
from skimage.measure import regionprops
import io
from PIL import Image
import nibabel as nib
import numpy as np

img_list = []
gt_list = []
baseline_pred_list = []
no_transformer_pred_list = []

names = [('patient020_frame11', 2), ('patient047_frame09', 4), ('patient085_frame09', 14), ('patient086_frame08', 5), ('patient085_frame01', 14), ('patient086_frame01', 2)]
fig, ax = plt.subplots(len(names), 4, figsize=(7, 8))
#fig, ax = plt.subplots(4, len(names), figsize=(8, 5))
for idx, payload in enumerate(names):
    name, index = payload
    baseline_path_pred = os.path.join(r'ACDC_output\Baseline\cv_niftis_raw', name + '.nii.gz')
    no_transformer_path_pred = os.path.join(r'ACDC_output\no_transformer\cv_niftis_raw', name + '.nii.gz')

    path_gt = os.path.join(r'ACDC_output\Baseline\gt_niftis', name + '.nii.gz')
    path_img = os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\imagesTr', name + '_0000.nii.gz')

    data_baseline = nib.load(baseline_path_pred)
    data_no_transformer = nib.load(no_transformer_path_pred)
    data_gt = nib.load(path_gt)
    data_img = nib.load(path_img)

    baseline_pred = data_baseline.get_fdata()
    no_transformer_pred = data_no_transformer.get_fdata()
    gt = data_gt.get_fdata()
    img = data_img.get_fdata()

    label = np.logical_or(gt[:, :, index] > 0, baseline_pred[:, :, index] > 0)
    label = np.logical_or(label, no_transformer_pred[:, :, index] > 0).astype(np.uint8)
    regions = regionprops(label)
    assert len(regions) == 1
    min_row, min_col, max_row, max_col = regions[0].bbox

    #for j in range(img.shape[-1]):
    #    print(j)
    #    fig, ax = plt.subplots(1, 4)
    #    ax[0].imshow(img[:, :, j], cmap='gray')
    #    ax[1].imshow(gt[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[2].imshow(baseline_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[3].imshow(no_transformer_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    plt.show()
    #    plt.waitforbuttonpress()
    #    plt.close(fig)
    #print('************************************')

    img = img[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    gt = gt[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    baseline_pred = baseline_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    no_transformer_pred = no_transformer_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]

    img_list.append(img)
    gt_list.append(gt)
    baseline_pred_list.append(baseline_pred)
    no_transformer_pred_list.append(no_transformer_pred)

row_lengths = [x.shape[0] for x in img_list]
col_lengths = [x.shape[1] for x in img_list]
max_row = max(row_lengths)
max_col = max(col_lengths)

for i in range(len(img_list)):
    current_img = img_list[i]
    current_gt = gt_list[i]
    current_baseline_pred = baseline_pred_list[i]
    current_no_transformer_pred = no_transformer_pred_list[i]

    pad_col = max_col - current_img.shape[1]
    pad_row = max_row - current_img.shape[0]

    if pad_col % 2 == 0:
        pad_left = pad_col // 2
        pad_right = pad_col // 2
    else:
        pad_left = pad_col // 2 + 1
        pad_right = pad_col // 2

    if pad_row % 2 == 0:
        pad_top = pad_row // 2
        pad_bottom = pad_row // 2
    else:
        pad_top = pad_row // 2 + 1
        pad_bottom = pad_row // 2

    pad_width = ((pad_top, pad_bottom), (pad_left, pad_right))
    current_img = np.pad(current_img, pad_width=pad_width)
    current_gt = np.pad(current_gt, pad_width=pad_width)
    current_baseline_pred = np.pad(current_baseline_pred, pad_width=pad_width)
    current_no_transformer_pred = np.pad(current_no_transformer_pred, pad_width=pad_width)

    #current_img = current_img
    #current_gt = current_gt
    #current_baseline_pred = current_baseline_pred
    #current_no_transformer_pred = current_no_transformer_pred
    
    ax[i, 0].imshow(current_img, cmap='gray')
    ax[i, 0].axis('off')
    ax[i, 1].imshow(current_gt, cmap='gray', vmin=0, vmax=3)
    ax[i, 1].axis('off')
    ax[i, 2].imshow(current_baseline_pred, cmap='gray', vmin=0, vmax=3)
    ax[i, 2].axis('off')
    ax[i, 3].imshow(current_no_transformer_pred, cmap='gray', vmin=0, vmax=3)
    ax[i, 3].axis('off')

    #ax[0, idx].imshow(img, cmap='gray')
    #ax[0, idx].axis('off')
    #ax[1, idx].imshow(gt, cmap='gray', vmin=0, vmax=3)
    #ax[1, idx].axis('off')
    #ax[2, idx].imshow(baseline_pred, cmap='gray', vmin=0, vmax=3)
    #ax[2, idx].axis('off')
    #ax[3, idx].imshow(no_transformer_pred, cmap='gray', vmin=0, vmax=3)
    #ax[3, idx].axis('off')

    plt.tight_layout()
#plt.savefig("visual_ablation_study_2.png", dpi=600)

png1 = io.BytesIO()
plt.savefig(png1, format="png", dpi=600)

png2 = Image.open(png1)

png2.save("Image_5.tiff", dpi=(600, 600))
png1.close()

In [9]:
%matplotlib qt

import os
import matplotlib.pyplot as plt
from skimage.measure import regionprops
import io
from PIL import Image
import nibabel as nib
import numpy as np
import cv2 as cv

img_list = []
gt_list = []
baseline_pred_list = []
no_transformer_pred_list = []

names = [('patient020_frame11', 2), ('patient047_frame09', 4), ('patient085_frame09', 14), ('patient086_frame08', 5), ('patient085_frame01', 14), ('patient086_frame01', 2)]
fig, ax = plt.subplots(len(names), 3, figsize=(6, 8))
#fig, ax = plt.subplots(4, len(names), figsize=(8, 5))
for idx, payload in enumerate(names):
    name, index = payload
    baseline_path_pred = os.path.join(r'ACDC_output\Baseline\cv_niftis_raw', name + '.nii.gz')
    no_transformer_path_pred = os.path.join(r'ACDC_output\no_transformer\cv_niftis_raw', name + '.nii.gz')

    path_gt = os.path.join(r'ACDC_output\Baseline\gt_niftis', name + '.nii.gz')
    path_img = os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\imagesTr', name + '_0000.nii.gz')

    data_baseline = nib.load(baseline_path_pred)
    data_no_transformer = nib.load(no_transformer_path_pred)
    data_gt = nib.load(path_gt)
    data_img = nib.load(path_img)

    baseline_pred = data_baseline.get_fdata()
    no_transformer_pred = data_no_transformer.get_fdata()
    gt = data_gt.get_fdata()
    img = data_img.get_fdata()

    label = np.logical_or(gt[:, :, index] > 0, baseline_pred[:, :, index] > 0)
    label = np.logical_or(label, no_transformer_pred[:, :, index] > 0).astype(np.uint8)
    regions = regionprops(label)
    assert len(regions) == 1
    min_row, min_col, max_row, max_col = regions[0].bbox

    #for j in range(img.shape[-1]):
    #    print(j)
    #    fig, ax = plt.subplots(1, 4)
    #    ax[0].imshow(img[:, :, j], cmap='gray')
    #    ax[1].imshow(gt[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[2].imshow(baseline_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    ax[3].imshow(no_transformer_pred[:, :, j], cmap='gray', vmin=0, vmax=3)
    #    plt.show()
    #    plt.waitforbuttonpress()
    #    plt.close(fig)
    #print('************************************')

    img = img[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    gt = gt[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    baseline_pred = baseline_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]
    no_transformer_pred = no_transformer_pred[min_row - 10:max_row + 10, min_col - 10:max_col + 10, index]

    img_list.append(img)
    gt_list.append(gt)
    baseline_pred_list.append(baseline_pred)
    no_transformer_pred_list.append(no_transformer_pred)

row_lengths = [x.shape[0] for x in img_list]
col_lengths = [x.shape[1] for x in img_list]
max_row = max(row_lengths)
max_col = max(col_lengths)

for i in range(len(img_list)):
    current_img = img_list[i]
    current_gt = gt_list[i]
    current_baseline_pred = baseline_pred_list[i]
    current_no_transformer_pred = no_transformer_pred_list[i]

    pad_col = max_col - current_img.shape[1]
    pad_row = max_row - current_img.shape[0]

    if pad_col % 2 == 0:
        pad_left = pad_col // 2
        pad_right = pad_col // 2
    else:
        pad_left = pad_col // 2 + 1
        pad_right = pad_col // 2

    if pad_row % 2 == 0:
        pad_top = pad_row // 2
        pad_bottom = pad_row // 2
    else:
        pad_top = pad_row // 2 + 1
        pad_bottom = pad_row // 2

    pad_width = ((pad_top, pad_bottom), (pad_left, pad_right))
    current_img = np.pad(current_img, pad_width=pad_width)
    current_gt = np.pad(current_gt, pad_width=pad_width)
    current_baseline_pred = np.pad(current_baseline_pred, pad_width=pad_width)
    current_no_transformer_pred = np.pad(current_no_transformer_pred, pad_width=pad_width)

    col_img_list = []
    for arr in [current_gt, current_baseline_pred, current_no_transformer_pred]:
        current_img_arr = np.copy(current_img)
        current_img_arr = cv.normalize(current_img_arr, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX).astype(np.uint8)
        current_img_arr = cv.cvtColor(current_img_arr, cv.COLOR_GRAY2RGB)
        for j in range(1, 4):
            seg = (arr == j).astype(np.uint8)
            contours, hierarchy = cv.findContours(seg, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            color = [0, 0, 0]
            color[j - 1] = 255
            cv.drawContours(current_img_arr, contours, -1, color, 1)
        col_img_list.append(current_img_arr)

    #current_img = current_img
    #current_gt = current_gt
    #current_baseline_pred = current_baseline_pred
    #current_no_transformer_pred = current_no_transformer_pred
    
    ax[i, 0].imshow(col_img_list[0], cmap='gray')
    ax[i, 0].axis('off')
    ax[i, 1].imshow(col_img_list[1], cmap='gray', vmin=0, vmax=3)
    ax[i, 1].axis('off')
    ax[i, 2].imshow(col_img_list[2], cmap='gray', vmin=0, vmax=3)
    ax[i, 2].axis('off')

    #ax[0, idx].imshow(img, cmap='gray')
    #ax[0, idx].axis('off')
    #ax[1, idx].imshow(gt, cmap='gray', vmin=0, vmax=3)
    #ax[1, idx].axis('off')
    #ax[2, idx].imshow(baseline_pred, cmap='gray', vmin=0, vmax=3)
    #ax[2, idx].axis('off')
    #ax[3, idx].imshow(no_transformer_pred, cmap='gray', vmin=0, vmax=3)
    #ax[3, idx].axis('off')

    plt.tight_layout()
plt.savefig("Transformer_vs_conv.png", dpi=600)

#png1 = io.BytesIO()
#plt.savefig(png1, format="png", dpi=600)
#
#png2 = Image.open(png1)
#
#png2.save("Image_6.tiff", dpi=(600, 600))
#png1.close()

In [1]:
import json
import os
import csv
import pandas as pd

df_disease = pd.read_excel(r'disease_saud\radiomics_all.xlsx')
payload_disease = df_disease.loc[df_disease['Descriptor'] == 'Infarction_Ratio']

results_list = []
#with open(r'Quorum_output_RV_removed\Baseline\temp_allClasses\summary.json', 'r') as fd_in:
with open(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\rubbish\fold_0\temp_allClasses\summary.json", 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    for data_dict in metric_file:
        name = data_dict['center']
        phase = data_dict['reference'].split('\\')[-1][11:13]
        infraction_percent = payload_disease
        center = name.split('-')[0]
        manufacturer = data_dict['manufacturer']

        #csv_path = os.path.join(r'data_saud_2\3D_RV_removed_2', phase.upper(), name, name + '_metadata.csv')
        csv_path = os.path.join(r'data_saud_2\3D_RV_removed', phase.upper(), name, name + '_metadata.csv')
        df = pd.read_csv(csv_path)

        if name in payload_disease:
            infraction_percent = payload_disease[name][0]
        else:
            infraction_percent = ''

        field_strength = str(data_dict['strength'])

        rv_dice = data_dict['1']['Dice']
        myo_dice = data_dict['2']['Dice']
        lv_dice = data_dict['3']['Dice']
        dice = (rv_dice + myo_dice + lv_dice) / 3

        rv_hd = data_dict['1']['Hausdorff Distance']
        myo_hd = data_dict['2']['Hausdorff Distance']
        lv_hd = data_dict['3']['Hausdorff Distance']
        hd = (rv_hd + myo_hd + lv_hd) / 3

        rv_assd = data_dict['1']['Avg. Symmetric Surface Distance']
        myo_assd = data_dict['2']['Avg. Symmetric Surface Distance']
        lv_assd = data_dict['3']['Avg. Symmetric Surface Distance']
        assd = (rv_assd + myo_assd + lv_assd) / 3

        results_list.append({'Name': name, 
                            'Center': center, 
                            'Manufacturer': manufacturer, 
                            'Phase': phase, 
                            'infraction_percent': infraction_percent,
                            'Field Strength': field_strength,
                            'RV dice': rv_dice,
                            'MYO dice': myo_dice,
                            'LV dice': lv_dice,
                            'Mean dice': dice, 
                            'RV HD': rv_hd,
                            'MYO HD': myo_hd,
                            'LV HD': lv_hd,
                            'Mean HD': hd,
                            'RV_assd': rv_assd,
                            'MYO_assd': myo_assd,
                            'LV_assd': lv_assd,
                            'Mean assd': assd
                            })

with open(os.path.join('rubbish', 'fold_0', 'Criteria_jmp_cross_dataset.csv'), 'w') as fd_csv:
    writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
    writer.writeheader() 
    writer.writerows(results_list) 

In [1]:
import json
import os
import csv
import pandas as pd

results_list = []
#with open(r'Quorum_output_RV_removed\Baseline\temp_allClasses\summary.json', 'r') as fd_in:
with open(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\Medis_to_ACDC\cv_niftis_raw\summary.json", 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    for data_dict in metric_file:
        phase = os.path.basename(data_dict['reference']).split('.')[0][-2:]
        name = os.path.basename(data_dict['reference']).split('_')[0]

        rv_dice = data_dict['1']['Dice']
        myo_dice = data_dict['2']['Dice']
        lv_dice = data_dict['3']['Dice']
        dice = (rv_dice + myo_dice + lv_dice) / 3

        rv_hd = data_dict['1']['Hausdorff Distance']
        myo_hd = data_dict['2']['Hausdorff Distance']
        lv_hd = data_dict['3']['Hausdorff Distance']
        hd = (rv_hd + myo_hd + lv_hd) / 3

        rv_assd = data_dict['1']['Avg. Symmetric Surface Distance']
        myo_assd = data_dict['2']['Avg. Symmetric Surface Distance']
        lv_assd = data_dict['3']['Avg. Symmetric Surface Distance']
        assd = (rv_assd + myo_assd + lv_assd) / 3

        results_list.append({'Name': name,
                            'Phase': phase, 
                            'RV dice': rv_dice,
                            'MYO dice': myo_dice,
                            'LV dice': lv_dice,
                            'Mean dice': dice, 
                            'RV HD': rv_hd,
                            'MYO HD': myo_hd,
                            'LV HD': lv_hd,
                            'Mean HD': hd,
                            'RV_assd': rv_assd,
                            'MYO_assd': myo_assd,
                            'LV_assd': lv_assd,
                            'Mean assd': assd
                            })

with open(os.path.join(r'C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\Medis_to_ACDC', 'metrics.csv'), 'w') as fd_csv:
    writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
    writer.writeheader() 
    writer.writerows(results_list) 

In [23]:
import pickle
from glob import glob
import pandas as pd

with open(r'splits_final.pkl', 'rb') as fd:
    data = pickle.load(fd)
    train_names = data[0]['train']
    train_names = [x[:-3] for x in train_names]

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

centers = {}
manufacturers = {}
strengths = {}
depths = {}
phases = {}
path_list = glob(r'custom_quorum\**\*.csv')
for path in path_list:
    df = pd.read_csv(path)
    filename = path.split('\\')[-1]
    patient_name = path.split('\\')[-2]
    phase = filename[:2]
    manufacturer = df['Manufacturer'].iloc[0]
    strength = df['Field Strength'].iloc[0]

    if patient_name in train_names:
        manufacturers = update_dict(manufacturers, str(manufacturer), filename)
        strengths = update_dict(strengths, str(strength), filename)
        phases = update_dict(phases, str(phase), filename)

for current_dict in [manufacturers, strengths, phases]:
    for k in current_dict.keys():
        print(f'{k} = {len(current_dict[k])}')

siemens = 248
philips = 14
ge = 48
1.5 = 280
3.0 = 30
ed = 155
es = 155


In [1]:
from glob import glob
import os
import json
import numpy as np
import yaml
import pandas as pd

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

output_file_name = 'quorum_output_field.yaml'
if os.path.exists(output_file_name):
    os.remove(output_file_name)

centers = {}
manufacturers = {}
strengths = {}
depths = {}
phases = {}

path_list = glob(r'Quorum_output_2\only_sfb\fold_0\temp_allClasses\*.gz')
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    center = df['Name'].iloc[0].split('-')[0]
    manufacturer = df['Manufacturer'].iloc[0]
    strength = df['Field Strength'].iloc[0]

    centers = update_dict(centers, str(center), filename)
    manufacturers = update_dict(manufacturers, str(manufacturer), filename)
    strengths = update_dict(strengths, str(strength), filename)
    phases = update_dict(phases, str(phase), filename)

with open(r'Quorum_output_2\only_sfb\fold_0\temp_allClasses\summary.json', 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    results_dict = {}
    for current_dict, criteria_name in zip([centers, manufacturers, strengths, phases], ['Center', 'Manufacturer', 'Field Strength', 'Phase']):
        for key in current_dict.keys():
            print(f'{key} = {len(current_dict[key])}')
            current_values = current_dict[key]
            list_of_dict = [x for x in metric_file if x['reference'].split('/')[-1] in current_values]
            mean_dice_list = []
            mean_hausdorff_list = []
            for data_dict in list_of_dict:
                rv_dice = data_dict['1']['Dice']
                myo_dice = data_dict['2']['Dice']
                lv_dice = data_dict['3']['Dice']
                results_list.append({criteria_name: key, 'RV': rv_dice, 'MYO': myo_dice, 'LV': lv_dice, 'Mean': (rv_dice + myo_dice + lv_dice) / 3})
                mean_dice_list.append([data_dict['1']['Dice'], data_dict['2']['Dice'], data_dict['3']['Dice']])
                mean_hausdorff_list.append([data_dict['1']['Hausdorff Distance'], data_dict['2']['Hausdorff Distance'], data_dict['3']['Hausdorff Distance']])
            class_dice = np.stack(mean_dice_list, axis=0).mean(axis=0)
            class_hausdorff = np.stack(mean_hausdorff_list, axis=0).mean(axis=0)
            results_dict[key] = {'Hausdorff distance': class_hausdorff.tolist(), 
                                'Mean Hausdorff distance': float(class_hausdorff.mean()), 
                                'Dice score': class_dice.tolist(),
                                'Mean dice score': float(class_dice.mean())}

with open(output_file_name, 'w') as fd:
    yaml.safe_dump(results_dict, fd, default_flow_style=False)
    #for results_dict in results_dicts:
    #    for key in results_dict.keys():
    #        fd.write(key + ': ' + str(results_dict[key]) + '\n')
    #    fd.write('\n')


250001 = 10
250005 = 2
276004 = 2
276005 = 2
276008 = 2
348001 = 6
348002 = 6
348003 = 2
348004 = 8
348007 = 6
616003 = 4
616005 = 2
616010 = 2
703001 = 10
703003 = 2
703004 = 8
724002 = 2
724006 = 4
siemens = 62
philips = 4
ge = 14
1.5 = 66
3.0 = 14
ed = 40
es = 40


Get ED/ES volume

In [5]:
import nibabel as nib
import numpy as np

data = nib.load(r'quorum_output\validation_raw\patient003_ed.nii.gz')
zoom = data.header.get_zooms()
pixel_volume = np.prod(zoom)
print(pixel_volume)
arr = data.get_fdata()
nb_pixels = np.count_nonzero(arr == 1)
print(nb_pixels * pixel_volume)

13.234375
187504.625


In [2]:
#%matplotlib qt
#
#import nibabel as nib
#import numpy as np
#from scipy.ndimage import distance_transform_edt
#from scipy.interpolate import interpn
#import matplotlib.pyplot as plt
#
#def bwperim(bw, n=4):
#    """
#    perim = bwperim(bw, n=4)
#    Find the perimeter of objects in binary images.
#    A pixel is part of an object perimeter if its value is one and there
#    is at least one zero-valued pixel in its neighborhood.
#    By default the neighborhood of a pixel is 4 nearest pixels, but
#    if `n` is set to 8 the 8 nearest pixels will be considered.
#    Parameters
#    ----------
#      bw : A black-and-white image
#      n : Connectivity. Must be 4 or 8 (default: 8)
#    Returns
#    -------
#      perim : A boolean image
#    """
#
#    if n not in (4,8):
#        raise ValueError('mahotas.bwperim: n must be 4 or 8')
#    rows,cols = bw.shape
#
#    # Translate image by one pixel in all directions
#    north = np.zeros((rows,cols))
#    south = np.zeros((rows,cols))
#    west = np.zeros((rows,cols))
#    east = np.zeros((rows,cols))
#
#    north[:-1,:] = bw[1:,:]
#    south[1:,:]  = bw[:-1,:]
#    west[:,:-1]  = bw[:,1:]
#    east[:,1:]   = bw[:,:-1]
#    idx = (north == bw) & \
#          (south == bw) & \
#          (west  == bw) & \
#          (east  == bw)
#    if n == 8:
#        north_east = np.zeros((rows, cols))
#        north_west = np.zeros((rows, cols))
#        south_east = np.zeros((rows, cols))
#        south_west = np.zeros((rows, cols))
#        north_east[:-1, 1:]   = bw[1:, :-1]
#        north_west[:-1, :-1] = bw[1:, 1:]
#        south_east[1:, 1:]     = bw[:-1, :-1]
#        south_west[1:, :-1]   = bw[:-1, 1:]
#        idx &= (north_east == bw) & \
#               (south_east == bw) & \
#               (south_west == bw) & \
#               (north_west == bw)
#    return ~idx * bw
#
#def signed_bwdist(im):
#    '''
#    Find perim and return masked image (signed/reversed)
#    '''    
#    perimeter = bwperim(im)
#
#    distance_map = bwdist(perimeter)
#
#    im = -distance_map*np.logical_not(im) + distance_map*im
#    return im
#
#def bwdist(im):
#    '''
#    Find distance map of image
#    '''
#    dist_im = distance_transform_edt(1-im)
#    return dist_im
#
#def interp_shape(arr, new_depth):
#    '''
#    Interpolate between two contours
#
#    Input: top 
#            [X,Y] - Image of top contour (mask)
#           bottom
#            [X,Y] - Image of bottom contour (mask)
#           precision
#             float  - % between the images to interpolate 
#                Ex: num=0.5 - Interpolate the middle image between top and bottom image
#    Output: out
#            [X,Y] - Interpolated image at num (%) between top and bottom
#
#    '''
#    X, Y, Z = arr.shape
#
#    distance_arr = []
#    for i in range(Z):
#        distance_arr.append(signed_bwdist(arr[:, :, i]))
#    distance_arr = np.stack(distance_arr, axis=-1)
#
#    x = np.arange(0, X)
#    y = np.arange(0, Y)
#    z = np.arange(0, Z)
#    points = (x, y, z)
#
#    stop = Z-1
#
#    # create ndgrids
#    grid = np.mgrid[:X, :Y, 0:stop:(new_depth * 1j)]
#    xi = np.rollaxis(grid, 0, 4)
#    xi = xi.reshape((X * Y * new_depth, 3))
#
#    out = interpn(points, distance_arr, xi)
#    out = out.reshape((X, Y, new_depth))
#
#    # Threshold distmap to values above 0
#    out = out > 0
#
#    #fig, ax = plt.subplots(2, out.shape[-1])
#    #for t in range(out.shape[-1]):
#    #    if t < arr.shape[-1]:
#    #        ax[0, t].imshow(arr[:, :, t], cmap='gray')
#    #    ax[1, t].imshow(out[:, :, t], cmap='gray')
#    #plt.show()
#
#    #print(out.shape)
#
#    return out
#
#
#data = nib.load(r'data_saud_2\inference_2\patient030_ed.nii.gz')
#arr = data.get_fdata()
#arr = arr == 1
#
#X, Y, Z = arr.shape
##print(arr.shape)
## Run interpolation
#out = interp_shape(arr, Z+1)
##print(out.shape)
##fig, ax = plt.subplots(1, Z+1)
##for i in range(Z+1):
##    ax[i].imshow(out[:, :, i], cmap='gray')
##plt.show()

In [11]:
%matplotlib qt

import nibabel as nib
import numpy as np
from scipy.ndimage import distance_transform_edt
from scipy.interpolate import interpn
import matplotlib.pyplot as plt

def bwperim(bw, n=4):
    """
    perim = bwperim(bw, n=4)
    Find the perimeter of objects in binary images.
    A pixel is part of an object perimeter if its value is one and there
    is at least one zero-valued pixel in its neighborhood.
    By default the neighborhood of a pixel is 4 nearest pixels, but
    if `n` is set to 8 the 8 nearest pixels will be considered.
    Parameters
    ----------
      bw : A black-and-white image
      n : Connectivity. Must be 4 or 8 (default: 8)
    Returns
    -------
      perim : A boolean image
    """

    if n not in (4,8):
        raise ValueError('mahotas.bwperim: n must be 4 or 8')
    rows,cols = bw.shape

    # Translate image by one pixel in all directions
    north = np.zeros((rows,cols))
    south = np.zeros((rows,cols))
    west = np.zeros((rows,cols))
    east = np.zeros((rows,cols))

    north[:-1,:] = bw[1:,:]
    south[1:,:]  = bw[:-1,:]
    west[:,:-1]  = bw[:,1:]
    east[:,1:]   = bw[:,:-1]
    idx = (north == bw) & \
          (south == bw) & \
          (west  == bw) & \
          (east  == bw)
    if n == 8:
        north_east = np.zeros((rows, cols))
        north_west = np.zeros((rows, cols))
        south_east = np.zeros((rows, cols))
        south_west = np.zeros((rows, cols))
        north_east[:-1, 1:]   = bw[1:, :-1]
        north_west[:-1, :-1] = bw[1:, 1:]
        south_east[1:, 1:]     = bw[:-1, :-1]
        south_west[1:, :-1]   = bw[:-1, 1:]
        idx &= (north_east == bw) & \
               (south_east == bw) & \
               (south_west == bw) & \
               (north_west == bw)
    return ~idx * bw

def signed_bwdist(im):
    '''
    Find perim and return masked image (signed/reversed)
    '''    
    perimeter = bwperim(im)

    distance_map = bwdist(perimeter)

    im = -distance_map*np.logical_not(im) + distance_map*im
    return im

def bwdist(im):
    '''
    Find distance map of image
    '''
    dist_im = distance_transform_edt(1-im)
    return dist_im

def interp_shape(top, bottom, precision):
    '''
    Interpolate between two contours

    Input: top 
            [X,Y] - Image of top contour (mask)
           bottom
            [X,Y] - Image of bottom contour (mask)
           precision
             float  - % between the images to interpolate 
                Ex: num=0.5 - Interpolate the middle image between top and bottom image
    Output: out
            [X,Y] - Interpolated image at num (%) between top and bottom

    '''
    top = signed_bwdist(top)
    bottom = signed_bwdist(bottom)

    # row,cols definition
    r, c = top.shape

    # Reverse % indexing
    precision = 1+precision

    # rejoin top, bottom into a single array of shape (2, r, c)
    top_and_bottom = np.stack((top, bottom))

    # create ndgrids 
    points = (np.r_[0, 2], np.arange(r), np.arange(c))
    xi = np.rollaxis(np.mgrid[:r, :c], 0, 3).reshape((r*c, 2))
    xi = np.c_[np.full((r*c),precision), xi]

    # Interpolate for new plane
    out = interpn(points, top_and_bottom, xi)
    out = out.reshape((r, c))

    # Threshold distmap to values above 0
    out = out > 0

    return out


#data = nib.load(r'data_saud_2\inference_2\patient030_ed.nii.gz')
#arr = data.get_fdata()
#arr = arr == 1
#
#X, Y, Z = arr.shape
##print(arr.shape)
## Run interpolation
#out = interp_shape(arr, Z+1)
#print(out.shape)
#fig, ax = plt.subplots(1, Z+1)
#for i in range(Z+1):
#    ax[i].imshow(out[:, :, i], cmap='gray')
#plt.show()

Get only patient in test set

In [9]:
from glob import glob
import pandas as pd

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d


cut_prediction_path = r'Quorum_output_2\only_sfb\fold_0\temp_allClasses'
all_prediction_path = r'data_saud_2\inference'

data_dict_cut = {}
data_dict_all = {}

path_list = glob(os.path.join(cut_prediction_path, '*.gz'))
cut_prediction_names = []
for path in path_list:
    filename = path.split('\\')[-1]
    phase = filename[11:13]
    name = filename.split('_')[0]
    df = pd.read_csv(os.path.join('custom_quorum_2', name, phase + '_info.csv'))
    actual_name = df['Name'].to_numpy()[0]
    spacing = (df['Space Between Slices'] - df['Slice Thickness']).to_numpy()[0]
    cut_prediction_names.append(actual_name)
    if phase == 'ed':
        update_dict(data_dict_cut, 'ed', (path, spacing, actual_name))
    elif phase == 'es':
        update_dict(data_dict_cut, 'es', (path, spacing, actual_name))

csv_list = glob(os.path.join(all_prediction_path, '*.csv'))
for csv_path in csv_list:
    filename = csv_path.split('\\')[-1]
    phase = filename[11:13]
    df = pd.read_csv(csv_path)
    actual_name = df['Name'].to_numpy()[0]
    if actual_name in cut_prediction_names:
        spacing = (df['Space Between Slices'] - df['Slice Thickness']).to_numpy()[0]
        path = csv_path[:-4] + '.nii.gz'
        if phase == 'ed':
            update_dict(data_dict_all, 'ed', (path, spacing, actual_name))
        elif phase == 'es':
            update_dict(data_dict_all, 'es', (path, spacing, actual_name))

In [27]:
from glob import glob
import nibabel as nib
from tqdm import tqdm

nb_rv_removed = 0
right_names = []
path_list = glob(r'Quorum_output_RV_removed\Baseline\temp_allClasses\*.gz')
for path in path_list:
    right_names.append(path.split('\\')[-1])
    data = nib.load(path)
    arr = data.get_fdata()
    nb_rv_removed += np.count_nonzero(arr == 3)

print(right_names)

nb_base = 0
path_list = glob(r'Quorum_output_2\all_data\only_sfb\temp_allClasses\*.gz')
for path in path_list:
    data = nib.load(path)
    arr = data.get_fdata()
    nb_base += np.count_nonzero(arr == 3)

nb_gt_rv_removed = 0
path_list = glob(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task030_Quorum_RV_removed\labelsTr\*.gz')
for path in tqdm(path_list):
    if path.split('\\')[-1] not in right_names:
        continue
    data = nib.load(path)
    arr = data.get_fdata()
    nb_gt_rv_removed += np.count_nonzero(arr == 3)

nb_gt = 0
path_list = glob(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task029_Quorum\labelsTr\*.gz')
for path in tqdm(path_list):
    if path.split('\\')[-1] not in right_names:
        continue
    data = nib.load(path)
    arr = data.get_fdata()
    nb_gt += np.count_nonzero(arr == 3)

print(nb_rv_removed)
print(nb_gt_rv_removed)
print(nb_base)
print(nb_gt)

['patient003_ed.nii.gz', 'patient003_es.nii.gz', 'patient005_ed.nii.gz', 'patient005_es.nii.gz', 'patient011_ed.nii.gz', 'patient011_es.nii.gz', 'patient017_ed.nii.gz', 'patient017_es.nii.gz', 'patient018_ed.nii.gz', 'patient018_es.nii.gz', 'patient020_ed.nii.gz', 'patient020_es.nii.gz', 'patient024_ed.nii.gz', 'patient024_es.nii.gz', 'patient026_ed.nii.gz', 'patient026_es.nii.gz', 'patient030_ed.nii.gz', 'patient030_es.nii.gz', 'patient033_ed.nii.gz', 'patient033_es.nii.gz', 'patient038_ed.nii.gz', 'patient038_es.nii.gz', 'patient040_ed.nii.gz', 'patient040_es.nii.gz', 'patient046_ed.nii.gz', 'patient046_es.nii.gz', 'patient049_ed.nii.gz', 'patient049_es.nii.gz', 'patient054_ed.nii.gz', 'patient054_es.nii.gz', 'patient059_ed.nii.gz', 'patient059_es.nii.gz', 'patient064_ed.nii.gz', 'patient064_es.nii.gz', 'patient072_ed.nii.gz', 'patient072_es.nii.gz', 'patient073_ed.nii.gz', 'patient073_es.nii.gz', 'patient077_ed.nii.gz', 'patient077_es.nii.gz', 'patient086_ed.nii.gz', 'patient086_es.

100%|██████████| 390/390 [01:33<00:00,  4.16it/s]

847711
844363
700370
700857


In [3]:
import matplotlib.pyplot as plt

data = nib.load(r'Quorum_output_2\all_data\only_sfb\temp_allClasses\patient109_es.nii.gz')
pred1 = data.get_fdata()

data_rv_removed = nib.load(r'Quorum_output_RV_removed\Baseline\temp_allClasses\patient109_es.nii.gz')
pred_rv_removed = data_rv_removed.get_fdata()

data_img1 = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task029_Quorum\imagesTr\patient109_es_0000.nii.gz')
img1 = data_img1.get_fdata()

data_img2 = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task030_Quorum_RV_removed\imagesTr\patient109_es_0000.nii.gz')
img2 = data_img2.get_fdata()

data_label1 = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task029_Quorum\labelsTr\patient109_es.nii.gz')
label1 = data_label1.get_fdata()

data_label2 = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task030_Quorum_RV_removed\labelsTr\patient109_es.nii.gz')
label2 = data_label2.get_fdata()

print(img1.shape)
print(pred1.shape)
print(label1.shape)
print(img2.shape)
print(pred_rv_removed.shape)
print(label2.shape)

for i in range(max(img1.shape[-1], img1.shape[-2])):
    fig, ax = plt.subplots(1, 6)
    ax[0].imshow(img1[:, :, i], cmap='gray')
    ax[1].imshow(pred1[:, :, i], cmap='gray')
    ax[2].imshow(label1[:, :, i], cmap='gray')
    ax[3].imshow(img2[:, :, i], cmap='gray')
    ax[4].imshow(pred_rv_removed[:, :, i], cmap='gray')
    ax[5].imshow(label2[:, :, i], cmap='gray')
    plt.waitforbuttonpress()
    plt.show()
    plt.close(fig)

(256, 256, 13)
(256, 256, 13)
(256, 256, 13)
(256, 256, 15)
(256, 256, 15)
(256, 256, 15)


KeyboardInterrupt: 

In [3]:
import pandas as pd
import nibabel as nib
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import cv2 as cv
from skimage.morphology import binary_erosion

def get_volume(arr, zoom, spacing):
    pixel_size = np.prod(zoom)
    gap_size = np.prod((zoom[0], zoom[1], spacing))
    
    volume_ml_interp = 0
    for s in range(len(arr)):
        top = arr[s]
        assert np.all(np.isin(top, [0, 1]))
        volume_ml_interp += (pixel_size * top.sum()) / 1000
        if s < len(arr) - 1:
            bottom = arr[s + 1]
            assert np.all(np.isin(bottom, [0, 1]))
            interpolated = interp_shape(top, bottom, precision=0.5)
            volume_ml_interp += (gap_size * interpolated.sum()) / 1000
    
    return volume_ml_interp

df = pd.read_excel('Quorum_Qmass_12_07_2021.xlsx')
#df_keep = pd.read_csv(r'data_saud_2\keep.csv', converters={"Slices": lambda x: list(map(int, x.strip("[]").split(", ")))})

name_list = []
results_list = []
results_list_gt = []
#with open(r'Quorum_output_2\all_data\only_sfb\temp_allClasses\summary.json', 'r') as fd_in:
with open(r'cross_dataset\fold_0\temp_allClasses\summary.json', 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    for idx in tqdm(range(0, len(metric_file), 2)):
        spacing_ed = float(metric_file[idx]['spacing between slices']) - float(metric_file[idx]['slice thickness'])
        spacing_es = float(metric_file[idx+1]['spacing between slices']) - float(metric_file[idx+1]['slice thickness'])
        name1 = metric_file[idx]['center']
        name2 = metric_file[idx+1]['center']

        assert name1 == name2
        name_list.append(name1)

        path1 = metric_file[idx]['test']
        path2 = metric_file[idx+1]['test']
        filename1 = path1.split('\\')[-1]
        filename2 = path2.split('\\')[-1]
        #if name1 in flipped_list:
        #    data_ed = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename2))
        #    data_es = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename1))
        #else:
        #data_ed = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename1))
        #data_es = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename2))
        data_ed = nib.load(os.path.join(r'cross_dataset\fold_0\temp_allClasses', filename1))
        data_es = nib.load(os.path.join(r'cross_dataset\fold_0\temp_allClasses', filename2))
        data_ed_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task030_Quorum_RV_removed\labelsTr', filename1))
        data_es_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task030_Quorum_RV_removed\labelsTr', filename2))
        zoom = list(data_ed.header.get_zooms())
        zoom_gt = list(data_ed_gt.header.get_zooms())
        arr_ed = data_ed.get_fdata()
        arr_es = data_es.get_fdata()
        arr_ed_gt = data_ed_gt.get_fdata()
        arr_es_gt = data_es_gt.get_fdata()

        assert spacing_ed == spacing_es
        assert zoom == zoom_gt

        right_slices = np.arange(min(arr_es.shape[-1], arr_ed.shape[-1]))
        arr_ed_keep = arr_ed[:, :, right_slices]
        arr_es_keep = arr_es[:, :, right_slices]

        right_slices_gt = np.arange(min(arr_es_gt.shape[-1], arr_ed_gt.shape[-1]))
        arr_ed_keep_gt = arr_ed_gt[:, :, right_slices_gt]
        arr_es_keep_gt = arr_es_gt[:, :, right_slices_gt]

        assert arr_ed_keep.shape == arr_es_keep.shape == arr_ed_keep_gt.shape == arr_es_keep_gt.shape

        class_volume = {'Patient ID': name1}
        class_volume_gt = {'Patient ID': name1}
        for i, class_name in enumerate(['RV', 'MYO', 'LV'], 1):
            volume_ed = get_volume(arr_ed_keep == i, zoom, spacing_ed)
            volume_es = get_volume(arr_es_keep == i, zoom, spacing_es)
            volume_ed_gt = get_volume(arr_ed_keep_gt == i, zoom, spacing_ed)
            volume_es_gt = get_volume(arr_es_keep_gt == i, zoom, spacing_es)
            if class_name == 'MYO':
                volume_ed = volume_ed * 1.055
                volume_es = volume_es * 1.055
                volume_ed_gt = volume_ed_gt * 1.055
                volume_es_gt = volume_es_gt * 1.055
            else:
                class_volume[class_name + 'EF_pred'] = ((volume_ed - volume_es) / volume_ed) * 100
                class_volume[class_name + 'EV_pred'] = (volume_ed - volume_es)
                class_volume_gt[class_name + 'EF_gt'] = ((volume_ed_gt - volume_es_gt) / volume_ed_gt) * 100
                class_volume_gt[class_name + 'EV_gt'] = (volume_ed_gt - volume_es_gt)

            class_volume[class_name + 'EDV_pred'] = volume_ed
            class_volume[class_name + 'ESV_pred'] = volume_es
            class_volume_gt[class_name + 'EDV_gt'] = volume_ed_gt
            class_volume_gt[class_name + 'ESV_gt'] = volume_es_gt

        results_list.append(class_volume)
        results_list_gt.append(class_volume_gt)

df_pred = pd.DataFrame.from_records(results_list)
df_pred_gt = pd.DataFrame.from_records(results_list_gt)

new_df = df.loc[(df['Patient ID'].isin(name_list)) & (df['Study description'] == 'Baseline_MRI')]

out = pd.merge(new_df, df_pred, on='Patient ID')
out = pd.merge(out, df_pred_gt, on='Patient ID')

out.to_csv(r'cross_dataset\fold_0\volume.csv')
#out.to_csv(r'Quorum_output_2\volume.csv')

100%|██████████| 40/40 [10:08<00:00, 15.22s/it]


In [6]:
import pickle

with open(r'ACDC_output\Baseline\plans.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data.keys())
    print(data['plans_per_stage'])

dict_keys(['num_stages', 'num_modalities', 'modalities', 'normalization_schemes', 'dataset_properties', 'list_of_npz_files', 'original_spacings', 'original_sizes', 'preprocessed_data_folder', 'num_classes', 'all_classes', 'base_num_features', 'use_mask_for_norm', 'keep_only_largest_region', 'min_region_size_per_class', 'min_size_per_class', 'transpose_forward', 'transpose_backward', 'data_identifier', 'plans_per_stage', 'preprocessor_name'])
{0: {'batch_size': 1, 'num_pool_per_axis': [3, 3], 'patch_size': array([224, 224]), 'median_patient_size_in_voxels': array([  9, 249, 219]), 'current_spacing': array([10.        ,  1.48438001,  1.48438001]), 'original_spacing': array([10.        ,  1.48438001,  1.48438001]), 'pool_op_kernel_sizes': [[2, 2], [2, 2], [2, 2]], 'conv_kernel_sizes': [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]], 'do_dummy_2D_data_aug': False}}


In [25]:
import pandas as pd
import nibabel as nib
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import cv2 as cv
from skimage.morphology import binary_erosion

def get_volume(arr, zoom, spacing):
    pixel_size = np.prod(zoom)
    gap_size = np.prod((zoom[0], zoom[1], spacing))
    
    volume_ml_interp = 0
    for s in range(len(arr)):
        top = arr[s]
        assert np.all(np.isin(top, [0, 1]))
        volume_ml_interp += (pixel_size * top.sum()) / 1000
        if s < len(arr) - 1:
            bottom = arr[s + 1]
            assert np.all(np.isin(bottom, [0, 1]))
            interpolated = interp_shape(top, bottom, precision=0.5)
            volume_ml_interp += (gap_size * interpolated.sum()) / 1000
    
    return volume_ml_interp
#df_keep = pd.read_csv(r'data_saud_2\keep.csv', converters={"Slices": lambda x: list(map(int, x.strip("[]").split(", ")))})

results_list = []
results_list_gt = []
#with open(r'Quorum_output_2\all_data\only_sfb\temp_allClasses\summary.json', 'r') as fd_in:
with open(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses\summary.json", 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    for idx in tqdm(range(0, len(metric_file), 2)):
        spacing_ed = 0.0
        spacing_es = 0.0
        filename1 = os.path.basename(metric_file[idx]['reference'])
        filename2 = os.path.basename(metric_file[idx+1]['reference'])

        name1 = filename1.split('_')[0]
        name2 = filename2.split('_')[0]

        assert name1 == name2

        path1 = metric_file[idx]['test']
        path2 = metric_file[idx+1]['test']
        #if name1 in flipped_list:
        #    data_ed = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename2))
        #    data_es = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename1))
        #else:
        #data_ed = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename1))
        #data_es = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename2))
        data_ed = nib.load(os.path.join(r'C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses', filename1))
        data_es = nib.load(os.path.join(r'C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses', filename2))
        data_ed_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\labelsTr', filename1))
        data_es_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\labelsTr', filename2))
        zoom = list(data_ed.header.get_zooms())
        zoom_gt = list(data_ed_gt.header.get_zooms())
        arr_ed = data_ed.get_fdata()
        arr_es = data_es.get_fdata()
        arr_ed_gt = data_ed_gt.get_fdata()
        arr_es_gt = data_es_gt.get_fdata()

        assert spacing_ed == spacing_es
        assert zoom == zoom_gt

        right_slices = np.arange(min(arr_es.shape[-1], arr_ed.shape[-1]))
        arr_ed_keep = arr_ed[:, :, right_slices]
        arr_es_keep = arr_es[:, :, right_slices]

        right_slices_gt = np.arange(min(arr_es_gt.shape[-1], arr_ed_gt.shape[-1]))
        arr_ed_keep_gt = arr_ed_gt[:, :, right_slices_gt]
        arr_es_keep_gt = arr_es_gt[:, :, right_slices_gt]

        assert arr_ed_keep.shape == arr_es_keep.shape == arr_ed_keep_gt.shape == arr_es_keep_gt.shape

        class_volume = {'Patient ID': name1}
        class_volume_gt = {'Patient ID': name1}
        for i, class_name in enumerate(['RV', 'MYO', 'LV'], 1):
            volume_ed = get_volume(arr_ed_keep == i, zoom, spacing_ed)
            volume_es = get_volume(arr_es_keep == i, zoom, spacing_es)
            volume_ed_gt = get_volume(arr_ed_keep_gt == i, zoom, spacing_ed)
            volume_es_gt = get_volume(arr_es_keep_gt == i, zoom, spacing_es)
            if class_name == 'MYO':
                volume_ed = volume_ed * 1.055
                volume_es = volume_es * 1.055
                volume_ed_gt = volume_ed_gt * 1.055
                volume_es_gt = volume_es_gt * 1.055
            else:
                class_volume[class_name + 'EF_pred'] = ((volume_ed - volume_es) / volume_ed) * 100
                class_volume[class_name + 'EV_pred'] = (volume_ed - volume_es)
                class_volume_gt[class_name + 'EF_gt'] = ((volume_ed_gt - volume_es_gt) / volume_ed_gt) * 100
                class_volume_gt[class_name + 'EV_gt'] = (volume_ed_gt - volume_es_gt)

            class_volume[class_name + 'EDV_pred'] = volume_ed
            class_volume[class_name + 'ESV_pred'] = volume_es
            class_volume_gt[class_name + 'EDV_gt'] = volume_ed_gt
            class_volume_gt[class_name + 'ESV_gt'] = volume_es_gt

        results_list.append(class_volume)
        results_list_gt.append(class_volume_gt)

df_pred = pd.DataFrame.from_records(results_list)
df_pred_gt = pd.DataFrame.from_records(results_list_gt)

out = pd.merge(df_pred, df_pred_gt, on='Patient ID')

out.to_csv(r'ACDC_output\Baseline\volume.csv')
#out.to_csv(r'Quorum_output_2\volume.csv')

100%|██████████| 100/100 [03:32<00:00,  2.12s/it]


In [24]:
import pandas as pd
import nibabel as nib
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os
import cv2 as cv
from skimage.morphology import binary_erosion

def get_volume(arr, zoom, spacing):
    pixel_size = np.prod(zoom)
    gap_size = np.prod((zoom[0], zoom[1], spacing))
    
    volume_ml_interp = 0
    for s in range(len(arr)):
        top = arr[s]
        assert np.all(np.isin(top, [0, 1]))
        volume_ml_interp += (pixel_size * top.sum()) / 1000
        if s < len(arr) - 1:
            bottom = arr[s + 1]
            assert np.all(np.isin(bottom, [0, 1]))
            interpolated = interp_shape(top, bottom, precision=0.5)
            volume_ml_interp += (gap_size * interpolated.sum()) / 1000
    
    return volume_ml_interp
#df_keep = pd.read_csv(r'data_saud_2\keep.csv', converters={"Slices": lambda x: list(map(int, x.strip("[]").split(", ")))})

results_list = []
results_list_gt = []
#with open(r'Quorum_output_2\all_data\only_sfb\temp_allClasses\summary.json', 'r') as fd_in:
with open(r"C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses\summary.json", 'r') as fd_in:
    metric_file = json.load(fd_in)['results']['all']
    for idx in tqdm(range(0, len(metric_file), 2)):
        spacing_ed = 0.0
        spacing_es = 0.0
        filename1 = os.path.basename(metric_file[idx]['reference'])
        filename2 = os.path.basename(metric_file[idx+1]['reference'])

        name1 = filename1.split('_')[0]
        name2 = filename2.split('_')[0]

        assert name1 == name2

        path1 = metric_file[idx]['test']
        path2 = metric_file[idx+1]['test']
        #if name1 in flipped_list:
        #    data_ed = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename2))
        #    data_es = nib.load(os.path.join(r'Quorum_output_2\only_sfb\temp_allClasses', filename1))
        #else:
        #data_ed = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename1))
        #data_es = nib.load(os.path.join(r'Quorum_output_2\all_data\only_sfb\temp_allClasses', filename2))
        data_ed = nib.load(os.path.join(r'C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses', filename1))
        data_es = nib.load(os.path.join(r'C:\Users\Portal\Documents\Isensee\nnUNet\nnunet\ACDC_output\Baseline\temp_allClasses', filename2))
        data_ed_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\labelsTr', filename1))
        data_es_gt = nib.load(os.path.join(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task027_ACDC\labelsTr', filename2))
        zoom = list(data_ed.header.get_zooms())
        zoom_gt = list(data_ed_gt.header.get_zooms())
        arr_ed = data_ed.get_fdata()
        arr_es = data_es.get_fdata()
        arr_ed_gt = data_ed_gt.get_fdata()
        arr_es_gt = data_es_gt.get_fdata()

        assert spacing_ed == spacing_es
        assert zoom == zoom_gt

        right_slices = np.arange(min(arr_es.shape[-1], arr_ed.shape[-1]))
        arr_ed_keep = arr_ed[:, :, right_slices]
        arr_es_keep = arr_es[:, :, right_slices]

        right_slices_gt = np.arange(min(arr_es_gt.shape[-1], arr_ed_gt.shape[-1]))
        arr_ed_keep_gt = arr_ed_gt[:, :, right_slices_gt]
        arr_es_keep_gt = arr_es_gt[:, :, right_slices_gt]

        assert arr_ed_keep.shape == arr_es_keep.shape == arr_ed_keep_gt.shape == arr_es_keep_gt.shape

        with open(os.path.join('ACDC_training', name1, 'info.cfg')) as f:
            lines = f.readlines()
            height = float(lines[3].strip().split(' ')[-1])
            weight = float(lines[5].strip().split(' ')[-1])
        
        BSA = 0.007184 * (weight**0.425*height**0.725)
        print(BSA)

        class_volume = {'Patient ID': name1}
        class_volume_gt = {'Patient ID': name1}
        for i, class_name in enumerate(['RV', 'MYO', 'LV'], 1):
            volume_ed = get_volume(arr_ed_keep == i, zoom, spacing_ed)
            volume_es = get_volume(arr_es_keep == i, zoom, spacing_es)
            volume_ed_gt = get_volume(arr_ed_keep_gt == i, zoom, spacing_ed)
            volume_es_gt = get_volume(arr_es_keep_gt == i, zoom, spacing_es)
            if class_name == 'MYO':
                volume_ed = (volume_ed * 1.055) / BSA
                volume_es = (volume_es * 1.055) / BSA
                volume_ed_gt = (volume_ed_gt * 1.055) / BSA
                volume_es_gt = (volume_es_gt * 1.055) / BSA
            else:
                class_volume[class_name + 'EF_pred'] = (((volume_ed - volume_es) / volume_ed) * 100) / BSA
                class_volume[class_name + 'EV_pred'] = (volume_ed - volume_es) / BSA
                class_volume_gt[class_name + 'EF_gt'] = (((volume_ed_gt - volume_es_gt) / volume_ed_gt) * 100) / BSA
                class_volume_gt[class_name + 'EV_gt'] = ((volume_ed_gt - volume_es_gt)) / BSA

            class_volume[class_name + 'EDV_pred'] = volume_ed / BSA
            class_volume[class_name + 'ESV_pred'] = volume_es / BSA
            class_volume_gt[class_name + 'EDV_gt'] = volume_ed_gt / BSA
            class_volume_gt[class_name + 'ESV_gt'] = volume_es_gt / BSA

        results_list.append(class_volume)
        results_list_gt.append(class_volume_gt)

df_pred = pd.DataFrame.from_records(results_list)
df_pred_gt = pd.DataFrame.from_records(results_list_gt)

out = pd.merge(df_pred, df_pred_gt, on='Patient ID')

out.to_csv(r'ACDC_output\Baseline\volume.csv')
#out.to_csv(r'Quorum_output_2\volume.csv')

  0%|          | 0/100 [00:00<?, ?it/s]

2.182186062279962


  1%|          | 1/100 [00:02<03:23,  2.05s/it]

1.7318886979699173


  2%|▏         | 2/100 [00:04<03:28,  2.12s/it]

...

 98%|█████████▊| 98/100 [03:23<00:03,  1.91s/it]

1.996421022275045


 99%|█████████▉| 99/100 [03:26<00:01,  1.99s/it]

1.6934092071865368


100%|██████████| 100/100 [03:28<00:00,  2.09s/it]


In [6]:
import pandas as pd
import nibabel as nib
import numpy as np
import json
from tqdm import tqdm
from glob import glob
import os

def get_volume(arr_ed, arr_es, zoom):
    pixel_size = np.prod(zoom)

    class_nb_pixels_ed = arr_ed == i
    class_nb_pixels_es = arr_es == i

    volume_no_interp_ed = (pixel_size * class_nb_pixels_ed.sum()) / 1000
    volume_no_interp_es = (pixel_size * class_nb_pixels_es.sum()) / 1000

    volume_interp_ed = volume_no_interp_ed

    volume_interp_es = volume_no_interp_es
    
    return volume_no_interp_ed, volume_no_interp_es, volume_interp_ed, volume_interp_es

df = pd.read_excel('Quorum_Qmass_12_07_2021.xlsx')

name_list = []
results_list = []
path_list = glob(r'data_saud_2\inference_2\*.gz')
#path_list = glob(r'data_saud_2\inference\*.gz')
path_list = sorted(path_list)
test_names = os.listdir(r'Quorum_output_2\all_data\only_sfb\temp_allClasses')
for idx in tqdm(range(0, len(path_list), 2)):
    path1 = path_list[idx]
    path2 = path_list[idx+1]
    if path1.split('\\')[-1] not in test_names:
        continue
    csv_path1 = path1.replace('.nii.gz', '.csv')
    csv_path2 = path2.replace('.nii.gz', '.csv')
    patient_df1 = pd.read_csv(csv_path1)
    patient_df2 = pd.read_csv(csv_path2)
    
    name1 = patient_df1['Name'].iloc[0]
    name2 = patient_df2['Name'].iloc[0]

    assert name1 == name2
    name_list.append(name1)

    data_ed = nib.load(path1)
    data_es = nib.load(path2)
    zoom = list(data_ed.header.get_zooms())
    arr_ed = data_ed.get_fdata()
    arr_es = data_es.get_fdata()

    class_volume = {'Patient ID': name1}
    for i, class_name in enumerate(['RV', 'MYO', 'LV'], 1):
        volume_no_interp_ed, volume_no_interp_es, volume_interp_ed, volume_interp_es = get_volume(arr_ed, arr_es, zoom)

        class_volume[class_name + 'EF_pred'] = ((volume_no_interp_ed - volume_no_interp_es) / volume_no_interp_ed) * 100
        class_volume[class_name + 'EV_pred'] = (volume_no_interp_ed - volume_no_interp_es)

        volume_no_interp_ed, volume_no_interp_es, volume_interp_ed, volume_interp_es = get_volume(arr_ed, arr_es, zoom)

        if class_name == 'MYO':
            class_volume[class_name + 'EDV_pred'] = volume_no_interp_ed * 1.055
            class_volume[class_name + 'ESV_pred'] = volume_no_interp_es * 1.055
        else:
            class_volume[class_name + 'EDV_pred'] = volume_no_interp_ed
            class_volume[class_name + 'ESV_pred'] = volume_no_interp_es
    results_list.append(class_volume)

df_pred = pd.DataFrame.from_records(results_list)

new_df = df.loc[(df['Patient ID'].isin(name_list)) & (df['Study description'] == 'Baseline_MRI')]

out = pd.merge(new_df, df_pred, on='Patient ID')

out.to_csv(r'Quorum_output_2\volume.csv')

100%|██████████| 195/195 [00:01<00:00, 155.47it/s]


Get volume and diseases for jmp

In [10]:
%matplotlib qt

import nibabel as nib
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import csv

df_disease = pd.read_excel(r'disease_saud\radiomics_all.xlsx')
payload_disease = df_disease.loc[df_disease['Descriptor'] == 'Infarction_Ratio']

#path_list = glob(r'quorum_output_2\temp_allClasses\*.gz')

for data_dict, output_file_name in zip([data_dict_cut, data_dict_all], ['disease_volume_cut.csv', 'disease_volume_all.csv']):
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
    results = []
    for i, class_name in enumerate(tqdm(['RV', 'MYO', 'LV']), 1):
        for phase in data_dict.keys():
            list_of_tuple = data_dict[phase]
            for path, spacing, name in list_of_tuple:
                data = nib.load(path)
                zoom = list(data.header.get_zooms())
                arr = data.get_fdata()
                pixel_size = np.prod(zoom)

                new_depth = round((spacing * (arr.shape[-1] - 1)) / zoom[-1]) + arr.shape[-1]
                #print(arr.shape[-1])
                #print(new_depth)
                #print('******************************')

                class_nb_pixels = arr == i

                volume_no_interp = (pixel_size * class_nb_pixels.sum()) / 1000
                if class_name == 'MYO':
                    volume_no_interp = volume_no_interp * 1.055

                if new_depth > arr.shape[-1]:
                    arr_interpolated = interp_shape(class_nb_pixels, new_depth)
                    volume_interp = (pixel_size * arr_interpolated.sum()) / 1000
                    if class_name == 'MYO':
                        volume_interp = volume_interp * 1.055
                else:
                    volume_interp = volume_no_interp

                if name in payload_disease:
                    infraction_percent = payload_disease[name][0]
                else:
                    infraction_percent = ''

                results.append({'Phase': phase, 'Class': class_name, 'Volume': volume_no_interp, 'Interpolated_volume': volume_interp, 'Infraction_percent': infraction_percent})
            
        #if class_name != 'MYO':
        #    results[class_name].update({
        #        'Fraction d\'ejection': {'no_interpolation': ((results[class_name]['ed']['volume'] - results[class_name]['es']['volume']) / results[class_name]['ed']['volume']) * 100,
        #                                'interpolated': ((results[class_name]['ed']['interpolated_volume'] - results[class_name]['es']['interpolated_volume']) / results[class_name]['ed']['interpolated_volume']) * 100},
        #        'Volume d\'ejection': {'no_interpolation': (results[class_name]['ed']['volume'] - results[class_name]['es']['volume']),
        #                                'interpolated': (results[class_name]['ed']['interpolated_volume'] - results[class_name]['es']['interpolated_volume'])}
        #                                })          

    with open(os.path.join(r'Quorum_output_2', output_file_name), 'w') as fd:
        writer = csv.DictWriter(fd, fieldnames=list(results[0].keys()))
        writer.writeheader() 
        writer.writerows(results) 

100%|██████████| 3/3 [00:11<00:00,  3.72s/it]


Get disease and dice for jmp

In [ ]:
import json
import os
import csv
import pandas as pd

df = pd.read_excel(r'disease_saud\radiomics_all.xlsx')
payload = df.loc[df['Descriptor'] == 'Infarction_Ratio']

folders = [r'Quorum_output_2\only_sfb\fold_0']
for folder in folders:
    results_list = []
    with open(os.path.join(folder, r'temp_allClasses\summary.json')) as fd_json:
        data = json.load(fd_json)
        results = data['results']['all']
        for res in results:
            rv_dice = res['1']['Dice']
            myo_dice = res['2']['Dice']
            lv_dice = res['3']['Dice']
            patient_name = res['center']
            if patient_name in payload:
                infraction_percent = payload[patient_name][0]
            else:
                infraction_percent = ''
            results_list.append({'Patient': patient_name, 'Infraction_percent': infraction_percent, 'RV': rv_dice, 'MYO': myo_dice, 'LV': lv_dice, 'Mean': (rv_dice + myo_dice + lv_dice) / 3})

    with open(os.path.join(folder, 'disease_jmp.csv'), 'w') as fd_csv:
        writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
        writer.writeheader() 
        writer.writerows(results_list) 
        

Get jmp data

In [7]:
import json
import os
import csv

folders = [r'Quorum_output_2\all_data\only_sfb', r'Quorum_output_2\all_data\no_transformer', r'Quorum_output_2\all_data\no_sfb', r'Quorum_output_2\all_data\no_ds']
results_list = []
for folder in folders:
    with open(os.path.join(folder, r'validation_raw\summary.json')) as fd_json:
        data = json.load(fd_json)
        results = data['results']['all']
        for res in results:
            name = res['test'].split('\\')[-1].split('.')[0]
            rv_hd = res['1']['Hausdorff Distance']
            myo_hd = res['2']['Hausdorff Distance']
            lv_hd = res['3']['Hausdorff Distance']
            rv_dice = res['1']['Dice']
            myo_dice = res['2']['Dice']
            lv_dice = res['3']['Dice']
            rv_assd = res['1']['Avg. Symmetric Surface Distance']
            myo_assd = res['2']['Avg. Symmetric Surface Distance']
            lv_assd = res['3']['Avg. Symmetric Surface Distance']
            results_list.append({'Name': name, 
                                'Method': folder.split('\\')[-1], 
                                'RV_HD': rv_hd, 
                                'MYO_HD': myo_hd, 
                                'LV_HD': lv_hd, 
                                'Mean_HD': (rv_hd + myo_hd + lv_hd) / 3,
                                'RV_Dice': rv_dice,
                                'MYO_Dice': myo_dice,
                                'LV_Dice': lv_dice,
                                'Mean Dice': (rv_dice + myo_dice + lv_dice) / 3,
                                'RV_assd': rv_assd,
                                'MYO_assd': myo_assd,
                                'LV_assd': lv_assd,
                                'Mean assd': (rv_assd + myo_assd + lv_assd) / 3})

with open(os.path.join('Quorum_output_2', 'Quorum_methods_jmp.csv'), 'w') as fd_csv:
    writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
    writer.writeheader() 
    writer.writerows(results_list) 

Get dice for 'temp_allClasses' folder

In [1]:
import json
import os
import csv

results_list = []
with open(r'Quorum_output_2\only_sfb\fold_0\temp_allClasses\summary.json') as fd_json:
    data = json.load(fd_json)
    results = data['results']['all']
    for res in results:
        rv_dice = res['1']['Dice']
        myo_dice = res['2']['Dice']
        lv_dice = res['3']['Dice']
        results_list.append({'RV': rv_dice, 'MYO': myo_dice, 'LV': lv_dice, 'Mean': (rv_dice + myo_dice + lv_dice) / 3})

with open(os.path.join('Quorum_output_2\only_sfb', 'Quorum_postprocess_allClasses_jmp.csv'), 'w') as fd_csv:
    writer = csv.DictWriter(fd_csv, fieldnames=list(results_list[0].keys()))
    writer.writeheader() 
    writer.writerows(results_list) 

In [35]:
from glob import glob
import nibabel as nib
import json
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from evaluation.metrics import dice, hausdorff_distance
import cv2 as cv
import os
from tqdm import tqdm

path_memory = np.zeros(shape=(9,), dtype=object)
score_memory = np.full_like(path_memory, fill_value=0)
arr_memory = np.zeros(shape=(9,), dtype=object)
idx_memory = np.zeros(shape=(9,), dtype=np.uint8)
path_list = glob(r'ACDC_output\Baseline\temp_allClasses\*.gz')
for path in path_list:
    data = nib.load(path)
    arr = data.get_fdata()

    zoom = list(data.header.get_zooms())
    zoom = zoom[:-1]

    filename = path.split('\\')[-1].split('.')[0] 
    path_array_gt = os.path.join('out\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task027_ACDC\\labelsTr\\', filename + '.nii.gz')

    gt_data = nib.load(path_array_gt)
    gt_arr = gt_data.get_fdata()

    for j in range(arr.shape[-1]):
        score = hausdorff_distance(arr[:, :, j], gt_arr[:, :, j], voxel_spacing=zoom)
        if score > score_memory[0]:
            arr_memory[0] = arr[:, :, j]
            score_memory[0] = score
            path_memory[0] = path
            idx_memory[0] = j
            sorted_indices = score_memory.argsort()
            arr_memory = arr_memory[sorted_indices]
            score_memory = score_memory[sorted_indices]
            path_memory = path_memory[sorted_indices]
            idx_memory = idx_memory[sorted_indices]
print(score_memory)
h = 3
#w = int(round(h * (16/9)))
w = h

fig, ax = plt.subplots(h, w, figsize=(8, 8))
fig.tight_layout()
fig.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99, wspace=0.05, hspace=0.01)

for i in range(9):
    current_path = path_memory[i]
    current_idx = idx_memory[i]
    filename = current_path.split('\\')[-1].split('.')[0] 
    path_array_gt = os.path.join('out\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task027_ACDC\\labelsTr\\', filename + '.nii.gz')
    path_array_img = os.path.join('out\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task027_ACDC\\imagesTr\\', filename + '_0000.nii.gz')

    gt_data = nib.load(path_array_gt)
    gt_arr = gt_data.get_fdata()[:, :, current_idx]

    img_data = nib.load(path_array_img)
    img_arr = img_data.get_fdata()[:, :, current_idx]

    pred_arr = arr_memory[i]

    img_arr = cv.normalize(img_arr, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX).astype(np.uint8)
    img_arr = cv.cvtColor(img_arr, cv.COLOR_GRAY2RGB)

    for j in range(1, 4):
        pred = (pred_arr == j).astype(np.uint8)
        gt = (gt_arr == j).astype(np.uint8)
        pred_contours, hierarchy = cv.findContours(pred, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        gt_contours, hierarchy = cv.findContours(gt, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        pred_color = [255, 0, 0]
        gt_color = [0, 255, 0]
        img_arr = cv.drawContours(img_arr, pred_contours, -1, pred_color, 1)
        img_arr = cv.drawContours(img_arr, gt_contours, -1, gt_color, 1)

    ax[int(i//w), int(i%w)].imshow(img_arr)
    ax[int(i//w), int(i%w)].set_axis_off()

plt.savefig("squares.png", dpi=200)
plt.show()

[47.01147097773805 47.330550062705605 48.839061275274325
 49.033918182821814 51.262682340547975 55.518688973869814
 56.44497458808888 59.39555565233817 74.49475816552884]


In [94]:
data_gt = nib.load(r'custom_quorum_RV_removed\patient004\patient004_ed_gt.nii.gz')
arr_gt = data_gt.get_fdata()

data_img = nib.load(r'custom_quorum_RV_removed\patient004\patient004_ed.nii.gz')
arr_img = data_img.get_fdata()

fig, ax = plt.subplots(1, 2)
ax[0].imshow(arr_img[:, :, 4], cmap='gray')
ax[1].imshow(arr_gt[:, :, 4], cmap='gray')
ax[0].axis('off')
ax[1].axis('off')
fig.tight_layout()

In [5]:
%matplotlib qt

from glob import glob
import nibabel as nib
import json
import numpy as np
import matplotlib.pyplot as plt
from evaluation.metrics import dice, hausdorff_distance, avg_surface_distance_symmetric
import cv2 as cv
import os
from tqdm import tqdm
import matplotlib.lines as lines
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mtick
import math
from skimage.measure import regionprops
import io
from PIL import Image

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

def get_metric(path_list, path_list_gt, metric=None):
    path_list = sorted(path_list, key=lambda x:x.split('\\')[-1])
    path_list_gt = sorted(path_list_gt, key=lambda x:x.split('\\')[-1])

    out_dict = {}
    scores = []
    for path, path_gt in tqdm(zip(path_list, path_list_gt), total=len(path_list)):
        data = nib.load(path)
        arr = data.get_fdata()

        zoom = list(data.header.get_zooms())
        zoom = zoom[:-1]

        data_gt = nib.load(path_gt)
        arr_gt = data_gt.get_fdata()

        assert arr.shape == arr_gt.shape
        patient_scores = []
        for i in range(arr.shape[-1]):
            #fig, ax = plt.subplots(1, 2)
            #ax[0].imshow(arr[:, :, i], cmap='gray')
            #ax[1].imshow(arr_gt[:, :, i], cmap='gray')
            #plt.show()
            #plt.waitforbuttonpress()
            #plt.close(fig)

            current_pred = arr[:, :, i]
            current_gt = arr_gt[:, :, i]

            #current_pred = arr
            #current_gt = arr_gt

            class_score = []
            for j in range(1, 4):
                current_class_pred = current_pred == j
                current_gt_pred = current_gt == j
                if metric == 'dice':
                    score = dice(current_class_pred, current_gt_pred)
                elif metric == 'hd':
                    score = hausdorff_distance(current_class_pred, current_gt_pred, voxel_spacing=zoom)
                elif metric == 'assd':
                    score = avg_surface_distance_symmetric(current_class_pred, current_gt_pred, voxel_spacing=zoom)
                class_score.append(score)
            out_dict = update_dict(out_dict, key=i/arr.shape[-1], value=np.array(class_score))
            patient_scores.append(np.array(class_score))
        patient_class_score = np.stack(patient_scores, axis=0)
        patient_class_score = np.nanmean(patient_class_score, axis=0)
        scores.append(patient_class_score)
    scores = np.stack(scores, 0)
    scores = np.nanmean(scores, axis=0)

    x = [[], [], []]
    y = [[], [], []]
    for key in out_dict.keys():
        class_dice = np.stack(out_dict[key], axis=0)
        class_dice = np.nanmean(class_dice, axis=0)
        for i in range(3):
            if not math.isnan(class_dice[i]):
                x[i].append(key)
                y[i].append(class_dice[i])

    print(np.array(scores))
    return x, y

def arrowed_spines(ax, metric=None):

    xmin, xmax = ax.get_xlim() 
    ymin, ymax = ax.get_ylim()

    #print(xmin)
    #print(xmax)
    #print(ymin)
    #print(ymax)

    # removing the default axis on all sides:
    for side in ['bottom','right','top','left']:
        ax.spines[side].set_visible(False)

    # removing the axis ticks
    #plt.xticks([]) # labels 
    #plt.yticks([])
    ax.xaxis.set_ticks_position('none') # tick markers
    ax.yaxis.set_ticks_position('none')

    ax.spines["left"].set_position(("data", -0.05))
    ax.spines["bottom"].set_position(("data", 0.0))
    ax.set_xticks([0.0, 1.0])

    # get width and height of axes object to compute 
    # matching arrowhead length and width
    #dps = fig.dpi_scale_trans.inverted()
    #bbox = ax.get_window_extent().transformed(dps)
    #width, height = bbox.width, bbox.height

    # manual arrowhead width and length
    hw = 1./20.*(ymax-ymin) 
    hl = 1./20.*(xmax-xmin)
    lw = 1. # axis line width
    ohg = 0.0 # arrow overhang

    # compute matching arrowhead length and width
    #yhw = hw/(ymax-ymin)*(xmax-xmin)* height/width 
    #yhl = hl/(xmax-xmin)*(ymax-ymin)* width/height

    if metric == 'assd':
        head_width = 0.2
        head_length = 0.02
    elif metric == 'hd':
        head_width = 1
        head_length = 0.02

    # draw x and y axis
    ax.arrow(-0.05, 0, xmax-xmin, 0., fc='k', ec='k', lw = lw, 
             head_width=head_width, head_length=head_length, overhang = ohg, 
             length_includes_head= True, clip_on = False) 
    
    if metric == 'assd':
        head_width = 0.02
        head_length = 0.2
    elif metric == 'hd':
        head_width = 0.02
        head_length = 1

    ax.arrow(-0.05, 0, 0., ymax-ymin, fc='k', ec='k', lw = lw, 
             head_width=head_width, head_length=head_length, overhang = ohg, 
             length_includes_head= True, clip_on = False)
    
    ax.set_xlabel('Depth')
    if metric == 'assd':
        ax.set_ylabel('ASSD (mm)')
    elif metric == 'hd':
        ax.set_ylabel('Hausdorff distance (mm)')
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

def get_image_ready(path, index):
    filename = path.split('\\')[-1].split('.')[0] 
    patient_name = filename.split('_')[0]
    path_array_gt = os.path.join(os.path.join('ACDC_training', patient_name, filename + '_gt.nii.gz'))
    path_array_img = os.path.join(os.path.join('ACDC_training', patient_name, filename + '.nii.gz'))
    path_array_pred = os.path.join(os.path.join(r'ACDC_output\Baseline\temp_allClasses', filename + '.nii.gz'))

    gt_data = nib.load(path_array_gt)
    gt_arr = gt_data.get_fdata()[:, :, index]

    img_data = nib.load(path_array_img)
    img_arr = img_data.get_fdata()[:, :, index]

    pred_data = nib.load(path_array_pred)
    pred_arr = pred_data.get_fdata()[:, :, index]

    img_arr = cv.normalize(img_arr, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX).astype(np.uint8)
    img_arr = cv.cvtColor(img_arr, cv.COLOR_GRAY2RGB)

    label = np.logical_or(gt_arr > 0, pred_arr > 0).astype(np.uint8)
    regions = regionprops(label)
    assert len(regions) == 1
    min_row, min_col, max_row, max_col = regions[0].bbox

    for j in range(1, 4):
        pred = (pred_arr == j).astype(np.uint8)
        gt = (gt_arr == j).astype(np.uint8)
        pred_contours, hierarchy = cv.findContours(pred, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        gt_contours, hierarchy = cv.findContours(gt, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        pred_color = [255, 0, 0]
        gt_color = [0, 255, 0]
        img_arr = cv.drawContours(img_arr, pred_contours, -1, pred_color, 1)
        img_arr = cv.drawContours(img_arr, gt_contours, -1, gt_color, 1)
    
    #y1 = max(0, int(round(y - 64)))
    #y2 = min(gt_arr.shape[0], int(round(y + 64)))
    #x1 = max(0, int(round(x - 64)))
    #x2 = min(gt_arr.shape[1], int(round(x + 64)))
    img_arr = img_arr[min_row - 10:max_row + 10, min_col - 10:max_col + 10]
    
    return img_arr

h = 2
#w = int(round(h * (16/9)))
w = 4

nb = h * w

path_memory_assd = np.zeros(shape=(nb,), dtype=object)
score_memory_assd = np.full_like(path_memory_assd, fill_value=0)
arr_memory_assd = np.zeros(shape=(nb,), dtype=object)
idx_memory_assd = np.zeros(shape=(nb,), dtype=np.uint8)

path_memory_hd = np.zeros(shape=(nb,), dtype=object)
score_memory_hd = np.full_like(path_memory_hd, fill_value=0)
arr_memory_hd = np.zeros(shape=(nb,), dtype=object)
idx_memory_hd = np.zeros(shape=(nb,), dtype=np.uint8)

path_list = glob(r'ACDC_output\Baseline\temp_allClasses\*.gz')
score_list = []
for path in path_list:
    data = nib.load(path)
    arr = data.get_fdata()

    zoom = list(data.header.get_zooms())
    zoom = zoom[:-1]

    filename = path.split('\\')[-1].split('.')[0] 
    patient_name = filename.split('_')[0]
    path_array_gt = os.path.join(os.path.join('ACDC_training', patient_name, filename + '_gt.nii.gz'))
    #path_array_gt = os.path.join(r'ACDC_training\patient_name\*_gt.nii.gz')
    #path_array_gt = os.path.join('out\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task027_ACDC\\labelsTr\\', filename + '.nii.gz')

    gt_data = nib.load(path_array_gt)
    gt_arr = gt_data.get_fdata()

    assert gt_arr.shape == arr.shape

    patient_score_assd = []
    patient_score_hd = []
    assd_merged_list = []
    hd_merged_list = []
    for j in range(arr.shape[-1]):
        class_assd_list = []
        class_hd_list = []
        assd_merged = avg_surface_distance_symmetric(arr[:, :, j], gt_arr[:, :, j], voxel_spacing=zoom)
        hd_merged = hausdorff_distance(arr[:, :, j], gt_arr[:, :, j], voxel_spacing=zoom)
        assd_merged_list.append(assd_merged)
        hd_merged_list.append(hd_merged)
        for c in range(1, 4):
            current_arr = arr[:, :, j] == c
            current_gt = gt_arr[:, :, j] == c
            class_assd = avg_surface_distance_symmetric(current_arr, current_gt, voxel_spacing=zoom)
            class_hd = hausdorff_distance(current_arr, current_gt, voxel_spacing=zoom)
            class_assd_list.append(class_assd)
            class_hd_list.append(class_hd)
        class_assd = np.array(class_assd_list)
        class_hd = np.array(class_hd_list)
        patient_score_assd.append(class_assd)
        patient_score_hd.append(class_hd)
    assd_merged_list = np.stack(assd_merged_list, axis=0)
    hd_merged_list = np.stack(hd_merged_list, axis=0)
    patient_score_assd = np.stack(patient_score_assd, axis=0)
    patient_score_hd = np.stack(patient_score_hd, axis=0)
    data_dict = {'scores_assd': patient_score_assd, 'scores_hd': patient_score_hd, 'assd_merged': assd_merged_list, 'hd_merged': hd_merged_list, 'depth': np.arange(arr.shape[-1]), 'path': np.full(shape=(arr.shape[-1],), fill_value=path)}
    score_list.append(data_dict)
        #assd = avg_surface_distance_symmetric(arr[:, :, j], gt_arr[:, :, j], voxel_spacing=zoom)
        #hd = hausdorff_distance(arr[:, :, j], gt_arr[:, :, j], voxel_spacing=zoom)
        #if assd > score_memory_assd[0]:
        #    arr_memory_assd[0] = arr[:, :, j]
        #    score_memory_assd[0] = assd
        #    path_memory_assd[0] = path
        #    idx_memory_assd[0] = j
        #    sorted_indices = score_memory_assd.argsort()
        #    arr_memory_assd = arr_memory_assd[sorted_indices]
        #    score_memory_assd = score_memory_assd[sorted_indices]
        #    path_memory_assd = path_memory_assd[sorted_indices]
        #    idx_memory_assd = idx_memory_assd[sorted_indices]
        #if hd > score_memory_hd[0]:
        #    arr_memory_hd[0] = arr[:, :, j]
        #    score_memory_hd[0] = hd
        #    path_memory_hd[0] = path
        #    idx_memory_hd[0] = j
        #    sorted_indices = score_memory_hd.argsort()
        #    arr_memory_hd = arr_memory_hd[sorted_indices]
        #    score_memory_hd = score_memory_hd[sorted_indices]
        #    path_memory_hd = path_memory_hd[sorted_indices]
        #    idx_memory_hd = idx_memory_hd[sorted_indices]

fig = plt.figure(figsize=(6, 11))
#fig = plt.figure(figsize=(11, 6))
#gs = GridSpec(4, 6, figure=fig)
gs = GridSpec(6, 4, figure=fig)

scores = np.concatenate([x['scores_assd'] for x in score_list], axis=0)
#scores = np.concatenate([x['scores_hd'] for x in score_list], axis=0)
depths = np.concatenate([x['depth'] for x in score_list], axis=0)
paths = np.concatenate([x['path'] for x in score_list], axis=0)
depth_percent = np.concatenate([x['depth'] / (len(x['scores_assd']) - 1) for x in score_list], axis=0)
#depth_percent = np.concatenate([x['depth'] / (len(x['scores_hd']) - 1) for x in score_list], axis=0)

sort_indices_depth = np.argsort(depth_percent)
sorted_scores_depth = scores[sort_indices_depth]
sorted_depth_depth = depth_percent[sort_indices_depth]

ax = fig.add_subplot(gs[2:, :])

for i, (label, color) in enumerate(zip(['RV', 'MYO', 'LV'], ['r', 'g', 'b'])):
    mask = ~np.isnan(sorted_scores_depth[:, i])
    assert len(sorted_depth_depth[mask]) == len(sorted_scores_depth[mask, i])
    ax.scatter(sorted_depth_depth[mask], sorted_scores_depth[mask, i], c=color, label=label)
#ax.scatter(np.array(x_assd[0]), np.array(y_assd[0]), c='r', label='RV')
#ax.scatter(np.array(x_assd[1]), np.array(y_assd[1]), c='g', label='MYO')
#ax.scatter(np.array(x_assd[2]), np.array(y_assd[2]), c='b', label='LV')
ax.legend(loc=(0.44, 0.7))
arrowed_spines(ax, metric='assd')
#arrowed_spines(ax, metric='hd')
ax.axvline(x=0.30, c='black', linestyle='dashed', alpha=0.5)
ax.axvline(x=0.68, c='black', linestyle='dashed', alpha=0.5)
ax.text(x=0.10, y=28.00, s='Base', alpha=1.0)
ax.text(x=0.45, y=28.00, s='Middle', alpha=1.0)
ax.text(x=0.85, y=28.00, s='Apex', alpha=1.0)

#ax.set_xlim(left=-50)

#mask = ~np.isnan(np.nanmean(scores, axis=1))
#scores = scores[mask, :]
#depths = depths[mask]
#paths = paths[mask]
#sort_indices_score = np.argsort(np.nanmean(scores, axis=1))
#sorted_scores_score = scores[sort_indices_score]
#sorted_depths_score = depths[sort_indices_score]
#sorted_paths_score = paths[sort_indices_score]
#display_paths = sorted_paths_score[-nb:]
#display_depths = sorted_depths_score[-nb:]

merged_scores = np.concatenate([x['assd_merged'] for x in score_list], axis=0)
#merged_scores = np.concatenate([x['hd_merged'] for x in score_list], axis=0)
mask = ~np.isnan(merged_scores)
merged_scores = merged_scores[mask]
depths = depths[mask]
paths = paths[mask]
sort_indices_score = np.argsort(merged_scores)
sorted_scores_score = merged_scores[sort_indices_score]
sorted_depths_score = depths[sort_indices_score]
sorted_paths_score = paths[sort_indices_score]
display_paths = sorted_paths_score[-nb:]
display_depths = sorted_depths_score[-nb:]

for i in range(nb):
    ax = fig.add_subplot(gs[int(i%h), int(i//h)])
    assd_img = get_image_ready(display_paths[i], display_depths[i])

    ax.imshow(assd_img)
    ax.set_axis_off()

#path_list = glob(r'ACDC_output\Baseline\temp_allClasses\*.gz')
#path_list_names = [x.split('\\')[-1][:13] for x in path_list]
#path_list_gt = glob(r'ACDC_training\**\*_gt.nii.gz', recursive=True)
#path_list_gt = [x for x in path_list_gt if x.split('\\')[-1][:13] in path_list_names]
#assert len(path_list_gt) == len(path_list)
#
##x_dice, y_dice = get_metric(path_list, path_list_gt, metric='dice')
##x_hd, y_hd = get_metric(path_list, path_list_gt, metric='hd')
#x_assd, y_assd = get_metric(path_list, path_list_gt, metric='assd')



#fig.add_artist(lines.Line2D([0.5, 0.5], [0, 1], linewidth=4, color='black'))
fig.tight_layout()
plt.tight_layout()
#plt.savefig("squares_assd_2.png", dpi=1200)
#plt.show()

png1 = io.BytesIO()
plt.savefig(png1, format="png", dpi=600)

png2 = Image.open(png1)

png2.save("Image_4.tiff", dpi=(600, 600))
png1.close()

In [28]:
np.nanmean([np.nan, np.nan, np.nan])

C:\Users\Portal\AppData\Local\Temp/ipykernel_19664/2630023403.py:1: RuntimeWarning: Mean of empty slice
  np.nanmean([np.nan, np.nan, np.nan])


nan

Get images for worst predictions

In [106]:
data = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task029_Quorum\imagesTr\patient082_ed_0000.nii.gz')
arr = data.get_fdata()

data_gt = nib.load(r'out\nnUNet_raw_data_base\nnUNet_raw_data\Task029_Quorum\labelsTr\patient082_ed.nii.gz')
arr_gt = data_gt.get_fdata()

for i in range(arr.shape[-1]):
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(arr[:, :, i], cmap='gray')
    ax[1].imshow(arr_gt[:, :, i], cmap='gray')
    plt.waitforbuttonpress()
    plt.show()
    plt.close(fig)

KeyboardInterrupt: 

Get number of centers

In [2]:
from glob import glob
import pandas as pd
import numpy as np

center_list = []
path_list = glob(r'data_saud_2\3D\**\*.csv', recursive=True)
for path in path_list:
    df = pd.read_csv(path)
    center = df['Name'][0].split('-')[0]
    center_list.append(center)
center_list = np.array(center_list)
out = np.unique(center_list)
print(out)
print(out.size)

['250001' '250005' '276004' '276005' '276006' '276007' '276008' '348001'
 '348002' '348003' '348004' '348007' '616003' '616005' '616006' '616009'
 '616010' '616012' '703001' '703003' '703004' '724002' '724005' '724006']
24


Get dice per depth level

In [4]:
%matplotlib qt

from evaluation.metrics import dice, hausdorff_distance, avg_surface_distance_symmetric
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from glob import glob
import math
import numpy as np
import nibabel as nib
from tqdm import tqdm

def update_dict(d, key, value):
    if key not in d:
        d[key] = [value]
    else:
        d[key].append(value)
    return d

def get_metric(path_list, path_list_gt, metric=None):
    path_list = sorted(path_list, key=lambda x:x.split('\\')[-1])
    path_list_gt = sorted(path_list_gt, key=lambda x:x.split('\\')[-1])

    out_dict = {}
    scores = []
    for path, path_gt in tqdm(zip(path_list, path_list_gt), total=len(path_list)):
        data = nib.load(path)
        arr = data.get_fdata()

        zoom = list(data.header.get_zooms())
        zoom = zoom[:-1]

        data_gt = nib.load(path_gt)
        arr_gt = data_gt.get_fdata()

        assert arr.shape == arr_gt.shape
        patient_scores = []
        for i in range(arr.shape[-1]):
            #fig, ax = plt.subplots(1, 2)
            #ax[0].imshow(arr[:, :, i], cmap='gray')
            #ax[1].imshow(arr_gt[:, :, i], cmap='gray')
            #plt.show()
            #plt.waitforbuttonpress()
            #plt.close(fig)

            current_pred = arr[:, :, i]
            current_gt = arr_gt[:, :, i]

            #current_pred = arr
            #current_gt = arr_gt

            class_score = []
            for j in range(1, 4):
                current_class_pred = current_pred == j
                current_gt_pred = current_gt == j
                if metric == 'dice':
                    score = dice(current_class_pred, current_gt_pred)
                elif metric == 'hd':
                    score = hausdorff_distance(current_class_pred, current_gt_pred, voxel_spacing=zoom)
                elif metric == 'assd':
                    score = avg_surface_distance_symmetric(current_class_pred, current_gt_pred, voxel_spacing=zoom)
                class_score.append(score)
            out_dict = update_dict(out_dict, key=i/arr.shape[-1], value=np.array(class_score))
            patient_scores.append(np.array(class_score))
        patient_class_score = np.stack(patient_scores, axis=0)
        patient_class_score = np.nanmean(patient_class_score, axis=0)
        scores.append(patient_class_score)
    scores = np.stack(scores, 0)
    scores = np.nanmean(scores, axis=0)

    x = [[], [], []]
    y = [[], [], []]
    for key in out_dict.keys():
        class_dice = np.stack(out_dict[key], axis=0)
        class_dice = np.nanmean(class_dice, axis=0)
        for i in range(3):
            if not math.isnan(class_dice[i]):
                x[i].append(key)
                y[i].append(class_dice[i])

    print(np.array(scores))
    return x, y

def arrowed_spines(fig, ax, metric=None):

    xmin, xmax = ax.get_xlim() 
    ymin, ymax = ax.get_ylim()

    #print(xmin)
    #print(xmax)
    #print(ymin)
    #print(ymax)

    # removing the default axis on all sides:
    for side in ['bottom','right','top','left']:
        ax.spines[side].set_visible(False)

    # removing the axis ticks
    #plt.xticks([]) # labels 
    #plt.yticks([])
    ax.xaxis.set_ticks_position('none') # tick markers
    ax.yaxis.set_ticks_position('none')

    ax.spines["left"].set_position(("data", 0.0))
    ax.spines["bottom"].set_position(("data", 0.0))
    ax.set_xticks([0.0, 1.0])

    # get width and height of axes object to compute 
    # matching arrowhead length and width
    dps = fig.dpi_scale_trans.inverted()
    bbox = ax.get_window_extent().transformed(dps)
    width, height = bbox.width, bbox.height

    # manual arrowhead width and length
    hw = 1./20.*(ymax-ymin) 
    hl = 1./20.*(xmax-xmin)
    lw = 1. # axis line width
    ohg = 0.0 # arrow overhang

    # compute matching arrowhead length and width
    yhw = hw/(ymax-ymin)*(xmax-xmin)* height/width 
    yhl = hl/(xmax-xmin)*(ymax-ymin)* width/height

    if metric == 'assd':
        head_width = 0.2
        head_length = 0.02
    elif metric == 'hd':
        head_width = 1
        head_length = 0.02

    # draw x and y axis
    ax.arrow(0, 0, xmax-xmin, 0., fc='k', ec='k', lw = lw, 
             head_width=head_width, head_length=head_length, overhang = ohg, 
             length_includes_head= True, clip_on = False) 
    
    if metric == 'assd':
        head_width = 0.02
        head_length = 0.2
    elif metric == 'hd':
        head_width = 0.02
        head_length = 1

    ax.arrow(0, 0, 0., ymax-ymin, fc='k', ec='k', lw = lw, 
             head_width=head_width, head_length=head_length, overhang = ohg, 
             length_includes_head= True, clip_on = False)
    
    ax.set_xlabel('Depth')
    if metric == 'assd':
        ax.set_ylabel('ASSD')
    elif metric == 'hd':
        ax.set_ylabel('Hausdorff distance')
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

path_list = glob(r'ACDC_output\Baseline\temp_allClasses\*.gz')
path_list_names = [x.split('\\')[-1][:13] for x in path_list]
path_list_gt = glob(r'ACDC_training\**\*_gt.nii.gz', recursive=True)
path_list_gt = [x for x in path_list_gt if x.split('\\')[-1][:13] in path_list_names]
assert len(path_list_gt) == len(path_list)

#x_dice, y_dice = get_metric(path_list, path_list_gt, metric='dice')
#x_hd, y_hd = get_metric(path_list, path_list_gt, metric='hd')
x_assd, y_assd = get_metric(path_list, path_list_gt, metric='assd')

fig, ax = plt.subplots(1, 1, figsize=(7, 6))
#fig.subplots_adjust(left=0.05, right=0.99, bottom=0.05, top=0.99)

ax.scatter(np.array(x_assd[0]), np.array(y_assd[0]), c='r', label='RV')
ax.scatter(np.array(x_assd[1]), np.array(y_assd[1]), c='g', label='MYO')
ax.scatter(np.array(x_assd[2]), np.array(y_assd[2]), c='b', label='LV')
ax.legend(loc=(0.44, 0.7))
arrowed_spines(fig, ax, metric='assd')
ax.axvline(x=0.333333, c='black', linestyle='dashed', alpha=0.5)
ax.axvline(x=0.666666, c='black', linestyle='dashed', alpha=0.5)
ax.text(x=0.13, y=5.04, s='Base', alpha=0.5)
ax.text(x=0.46, y=5.04, s='Middle', alpha=0.5)
ax.text(x=0.80, y=5.04, s='Apex', alpha=0.5)

#ax[1].scatter(np.array(x_hd[0]), np.array(y_hd[0]), c='r', label='RV')
#ax[1].scatter(np.array(x_hd[1]), np.array(y_hd[1]), c='g', label='MYO')
#ax[1].scatter(np.array(x_hd[2]), np.array(y_hd[2]), c='b', label='LV')
#ax[1].legend(loc=(0.44, 0.7))
#arrowed_spines(fig, ax[1], metric='hd')
#ax[1].axvline(x=0.333333, c='black', linestyle='dashed', alpha=0.5)
#ax[1].axvline(x=0.666666, c='black', linestyle='dashed', alpha=0.5)
#ax[1].text(x=0.13, y=30.04, s='Base', alpha=0.5)
#ax[1].text(x=0.46, y=30.04, s='Middle', alpha=0.5)
#ax[1].text(x=0.80, y=30.04, s='Apex', alpha=0.5)

plt.tight_layout()

plt.savefig("depth_assd.png", dpi=200)
plt.show()

100%|██████████| 200/200 [00:22<00:00,  9.04it/s]
C:\Users\Portal\AppData\Local\Temp/ipykernel_18508/2623967950.py:74: RuntimeWarning: Mean of empty slice
  class_dice = np.nanmean(class_dice, axis=0)


[1.32893187 0.91881481 0.90285253]
